In [12]:
! pip install transformers
! pip install tf-keras
! pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 8.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 9.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 7.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 8.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.1 MB/s eta 0:00:0000:010:0

In [9]:
import pandas as pd
import os
import numpy as np
import random
import json
import re
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from transformers import pipeline
from collections import defaultdict

ModuleNotFoundError: No module named 'model'

In [292]:
import spacy
import re
from collections import Counter
import requests

class ColumnAnalysis:

    def __init__(self):
       
        self.entity_type_dict = {
            "PERSON": "NE",
            "NORP": "NE",
            "FAC": "NE",
            "ORG": "NE",
            "GPE": "NE",
            "LOC": "NE",
            "PRODUCT": "NE",
            "EVENT": "NE",
            "WORK_OF_ART": "NE",
            "LAW": "NE",
            "LANGUAGE": "NE",
            "DATE": "LIT",
            "TIME": "LIT",
            "PERCENT": "LIT",
            "MONEY": "LIT",
            "QUANTITY": "LIT",
            "ORDINAL": "LIT",
            "CARDINAL": "LIT",
            "URL": "LIT",
            "DESC": "LIT",
            "TOKEN": "NE",
            "INTEGER": "LIT",
            "FLOAT": "LIT",
            "DATETIME": "LIT",
            "ADDRESS": "LIT",
            "EMAIL": "LIT"
        }

        self.LIT_DATATYPE = {
            "DATE": "DATETIME", 
            "TIME": "STRING", 
            "PERCENT": "STRING", 
            "MONEY": "STRING", 
            "QUANTITY": "STRING", 
            "ORDINAL": "NUMBER", 
            "CARDINAL": "NUMBER", 
            "URL": "STRING",
            "DESC": "STRING",
            "TOKEN": "STRING",
            "INTEGER": "NUMBER",
            "FLOAT": "NUMBER",
            "DATETIME": "DATETIME",
            "ADDRESS": "STRING",
            "EMAIL": "STRING",
            "STRING": "STRING"
        }

        self.NE_DATATYPE = ["PERSON", "NORP", "FAC", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE"]


    def classifiy_columns(self, columns = []):

        def most_frequent_element(input_list):
            # Count occurrences of each element in the list
            counter = Counter(input_list)
            
            # Get the most common element(s) and their count
            most_common = counter.most_common(1)  # Returns a list of (element, count)
            
            # If there are ties (multiple elements with the same frequency), this will return the first one encountered.
            return most_common[0][0]  # Return the element from the first tuple

       
        def update_dict(dictionary, key, value=1):
            if key not in dictionary:
                dictionary[key] = 0
            dictionary[key] += value    

        url_pattern = re.compile(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$', re.IGNORECASE)
        email_pattern = re.compile(r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', re.IGNORECASE)
        address_pattern = re.compile(
            r'\d+\s+\w+\s+(?:street|st|avenue|ave|road|rd|boulevard|blvd|lane|ln|drive|dr|court|ct|circle|cir|place|pl)\.?\s*\w*', re.IGNORECASE)
        datetime_pattern = re.compile(
            r'(?:\d{4}-\d{2}-\d{2})'  # YYYY-MM-DD format
            r'|(?:31(?:\/|-|\.)0?[13578]|1[02](?:\/|-|\.)\d{4})'  # 31 days months
            r'|(?:29|30(?:\/|-|\.)0?[1,3-9]|1[0-2](?:\/|-|\.)\d{4})'  # 29/30 days months
            r'|(?:0?[1-9]|[12]\d|3[01])(?:\/|-|\.)'  # Day
            r'(?:0?[1-9]|1[0-2])(?:\/|-|\.)\d{4}'  # Month
            r'|(?:0?[1-9]|1[0-2])/(?:0?[1-9]|[12]\d|3[01])/(?:\d{2})'  # MM/DD/YY format
            r'|(?:0?[1-9]|1[0-2])/(?:0?[1-9]|[12]\d|3[01])/\d{2}'  # MM/DD/YY format
            r'\b\d{2}/(?:0?[1-9]|[12]\d|3[01])/(?:0?[1-9]|1[0-2])\b'  # YY/DD/MM format
            r'|(?:[01]?\d|2[0-3]):[0-5]\d\.[0-5]\d'  # HH:MM.SS format
            r'|(?:[01]?\d|2[0-3]):[0-5]\d'  # HH:MM format
            r'|(?:[0-5]?\d):[0-5]\d(?:\.\d{1,2})?'  # H:MM or H:MM.S format
            r'|(?:2[0-3]|[01]?\d)h[0-5]?\d(?:m[0-5]?\d(?:\.\d{1,2})?s)?',  # HhMMmSSs format
            re.IGNORECASE
        )

        
        
        #pattern = r'^[\d,.]+$'  # ^ asserts position at the start, [\d,.]+ matches one or more digits, commas, or periods, $ asserts position at the end.
        #return re.match(pattern, input_string) is not None

        col_type = []
        final_result = {}
        
        for col_name, col_data in df.items():
            
            # Analyze the concatenated text using Spacy
            labels = {}
            tags = {"NE": 0, "LIT": 0}
            type = []
            
            for cell in col_data:
                is_number = False
                label = None
            
                if isinstance(cell, str):
                    if cell == "NaN":
                        label = ""
                    
                    # Check if the cell is a URL
                    elif re.match(url_pattern, cell):
                        label = "URL"
                    
                    # Check if the cell is an Email
                    elif re.match(email_pattern, cell):
                        label = "EMAIL"
                    
                    # Check if the cell is an Address
                    elif re.match(address_pattern, cell):
                        label = "ADDRESS"
                        
                    # Check if the cell matches datetime pattern
                    elif re.match(datetime_pattern, cell):
                        label = "DATETIME"


                # Check if the cell is a number
                if label is None:
                    try:
                        cell = str(cell)
                        if ',' in cell or '.' in cell or '%' in cell or '$' in cell:
                            cell = cell.replace('.', '').replace(',', '').replace('%', '').replace('$', '')
                        
                        if len(cell) - len(re.findall(r'\d', cell)) < 5:
                            is_number = True
                    except:
                        pass
                
                
                if is_number:
                    label = "NUMBER"
                elif len(cell.split(" ")) >= 15:   #is the choosen treshold for discriminate descriptions and strings
                    label = "NOA"
                elif len(cell.split(" ")) == 1 and len(cell) <= 4:
                    label = "STRING"
                    
                if label is not None:
                    type.append(label)
                else:
                    # do the lookup
                    url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
                    params = {
                        'name': cell,
                        'token': 'lamapi_demo_2023',
                        'kg': 'wikidata',
                        'limit': 10,
                        'query': f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{cell}", "boost": 2.0}}}}}}]}}}}}}',
                        'sort': [
                            f'''{{"popularity": {{"order": "desc"}}}}'''
                        ]
                    }

                    response = requests.get(url, params=params)
                    if response.status_code == 200:
                        data = response.json()
                        if len(data) > 0:
                            print(f"{cell} --> NE_{data[0]['NERtype']}")
                            type.append(f"NE_{data[0]['NERtype']}")
                            break
                        else:
                            type.append("None")

            col_type.append(next((x for x in type if x != ""), None))
  
           
        return col_type


In [ ]:
tables = "./data/Dataset/Dataset/Round1_T2D/tables/"
#tables = "./data/Dataset/Dataset/Round3_2019/tables/"

target = []
column_analysis = ColumnAnalysis()

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)

    result = column_analysis.classifiy_columns(df.iloc[1:100])

    final = list(zip(df.iloc[2], df.columns, result))
    filtered_data = [item for item in final if item[-1] != 'None']
    print(final)
    print("____________________")

   


## Round1_T2D

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)

R1_cea = [item[0]for item in R1_sorted_mentions]

In [ ]:
categories = [
    "Place",
    "PopulatedPlace",
    "City",
    "Country",
    "Region",
    "Mountain",
    "Island",
    "Lake",
    "River",
    "Park",
    "Building",
    "HistoricPlace",
    "Monument",
    "Bridge",
    "Road",
    "Airport",
    "Person",
    "Artist",
    "Athlete",
    "Politician",
    "Scientist",
    "Writer",
    "Actor",
    "Musician",
    "MilitaryPerson",
    "Religious",
    "Royalty",
    "Criminal",
    "Organisation",
    "Company",
    "EducationalInstitution",
    "PoliticalParty",
    "SportsTeam",
    "Non-ProfitOrganisation",
    "GovernmentAgency",
    "ReligiousOrganisation",
    "Band",
    "Library",
    "Museum",
    "Hospital",
    "University",
    "TradeUnion"
]

# Mapping of subtypes to macro classes
mapping = {
    "Place": ["PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River", "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"],
    "Person": ["Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson", "Religious", "Royalty", "Criminal"],
    "Organisation": ["Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation", "GovernmentAgency", "ReligiousOrganisation", "Band"],
    "Institution": ["Library", "Museum", "Hospital", "University", "TradeUnion"]
}

In [ ]:
tables = "./data/Dataset/Dataset/Round1_T2D/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)

        
        # Check for NE flag
        NE_flag = column.isin(R1_cea).any()
        if NE_flag:
            joined_cells = column.str.cat(sep='-')        
            doc = nlp(joined_cells)
            entities = {"ORG": [], "PERS": [], "LOC": [], "OTHERS": []}

            # Extract entities and classify them
            for ent in doc.ents:
                if ent.label_ == "ORG":
                    entities["ORG"].append(ent.text)
                elif ent.label_ == "PERSON":
                    entities["PERS"].append(ent.text)
                elif ent.label_ == "GPE" or ent.label_ == "FAC":  # GPE (Geopolitical Entity)
                    entities["LOC"].append(ent.text)
                else:
                    entities["OTHERS"].append(ent.text)
            
            # Find the key of the longest entities list
            longest_key = max(entities, key=lambda k: len(entities[k]))
            print(f"{joined_cells} --> The key of the longest list is '{longest_key}'")
                    
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## Round3_2019

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round3_2019_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R3_sorted_mentions = json.load(file)

R3_cea = [item[0]for item in R3_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round3_2019/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R3_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## 2T_Round4

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/2T_Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_2T_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/2T_Round4/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_2T_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## Round4

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round4_2020/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## DF creation

In [ ]:
# Create the DataFrame
df_def = pd.DataFrame({
    'column names': columns,
    'median_lengths': median_lengths,
    'median_token_counts': median_token_counts,
    'average_numeric_counts': average_numeric_counts,
    'target': target
})

In [ ]:
df_def.to_csv('./data/NE_lit_dataset.csv', index=False)

In [ ]:
###################################
#   READ DIRECTLY THE DATASET HERE
###################################

df = pd.read_csv('./data/NE_lit_dataset.csv')
filtered_df = df[df['target'].isin(['lit', 'NE'])]

# Displaying the filtered DataFrame
df[:7]

In [ ]:
target_counts = df['target'].value_counts()

# Extract counts for specific values
ne_count = target_counts.get("NE", 0)
lit_count = target_counts.get("lit", 0)
none_count = df.shape[0] - (ne_count+lit_count)

print(f"Count of 'NE': {ne_count}")
print(f"Count of 'lit': {lit_count}")
print(f"Count of 'NaN': {none_count}")

## Model training

In [ ]:
# Convert the target variable to numeric
label_encoder = LabelEncoder()
filtered_df['target'] = label_encoder.fit_transform(filtered_df['target'])

# One-hot encode the 'column names' column
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_columns = one_hot_encoder.fit_transform(filtered_df[['column names']])

# Combine the encoded categorical data with the numeric data
numeric_data = filtered_df[['median_lengths', 'median_token_counts', 'average_numeric_counts']].values
X = np.hstack([encoded_columns, numeric_data])
y = filtered_df['target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

In [ ]:

# Make predictions
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)
mapped_predictions = ["lit" if pred == 1 else "NE" for pred in y_pred.ravel()]

# Extract the part of X_test that corresponds to the one-hot encoded columns
encoded_columns_test = X_test[:, :encoded_columns.shape[1]]

# Inverse transform the one-hot encoded columns to get the original categorical labels
original_labels = one_hot_encoder.inverse_transform(encoded_columns_test)

# Print a few examples to check
for i in range(100):
    print(f'Original label: {original_labels[i]}, Predicted: {mapped_predictions[i]}')
    


In [ ]:
filtered_df[:10]